In [15]:
import numpy as np
import pandas as pd
import json
import nltk
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import re
import math
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [17]:
def load_SNLI(path:str):
    with open(path, 'r') as f:
        # Read all lines in the file and parse them as JSON
        data = [json.loads(line.strip()) for line in f]
    return data

def load_SNLI_as_pandas(path:str):
    df = pd.read_json(path, lines=True)
    return df

In [9]:
#load the training data into a variable for future use
path_data = "SNLI_data/snli_1.0_train.jsonl"
snli_data = load_SNLI(path=path_data)

In [18]:
snli_data_pd = load_SNLI_as_pandas(path=path_data)

In [19]:
snli_data_pd.head(10)

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],3416050480.jpg#4,neutral,3416050480.jpg#4r1n,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,A person is training his horse for a competition.,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
1,[contradiction],3416050480.jpg#4,contradiction,3416050480.jpg#4r1c,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is at a diner, ordering an omelette.",( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
2,[entailment],3416050480.jpg#4,entailment,3416050480.jpg#4r1e,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is outdoors, on a horse.","( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
3,[neutral],2267923837.jpg#2,neutral,2267923837.jpg#2r1n,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,They are smiling at their parents,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...
4,[entailment],2267923837.jpg#2,entailment,2267923837.jpg#2r1e,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,There are children present,( There ( ( are children ) present ) ),(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...
5,[contradiction],2267923837.jpg#2,contradiction,2267923837.jpg#2r1c,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,The kids are frowning,( ( The kids ) ( are frowning ) ),(ROOT (S (NP (DT The) (NNS kids)) (VP (VBP are...
6,[contradiction],3691670743.jpg#0,contradiction,3691670743.jpg#0r1c,A boy is jumping on skateboard in the middle o...,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,The boy skates down the sidewalk.,( ( The boy ) ( ( ( skates down ) ( the sidewa...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ skate...
7,[entailment],3691670743.jpg#0,entailment,3691670743.jpg#0r1e,A boy is jumping on skateboard in the middle o...,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,The boy does a skateboarding trick.,( ( The boy ) ( ( does ( a ( skateboarding tri...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ does)...
8,[neutral],3691670743.jpg#0,neutral,3691670743.jpg#0r1n,A boy is jumping on skateboard in the middle o...,( ( A boy ) ( ( is ( ( jumping ( on skateboard...,(ROOT (S (NP (DT A) (NN boy)) (VP (VBZ is) (VP...,The boy is wearing safety equipment.,( ( The boy ) ( ( is ( wearing ( safety equipm...,(ROOT (S (NP (DT The) (NN boy)) (VP (VBZ is) (...
9,[neutral],4804607632.jpg#0,neutral,4804607632.jpg#0r1n,An older man sits with his orange juice at a s...,( ( An ( older man ) ) ( ( ( sits ( with ( ( h...,(ROOT (S (NP (DT An) (JJR older) (NN man)) (VP...,An older man drinks his juice as he waits for ...,( ( An ( older man ) ) ( ( ( drinks ( his juic...,(ROOT (S (NP (DT An) (JJR older) (NN man)) (VP...


### Preprocessing functions

In [38]:
# Here to implement all functions for preprocessing the given data
def remove_duplicates(df:pd.DataFrame, subset:list[str]) -> pd.DataFrame:
    df_preprocessed = df.drop_duplicates(subset=subset)

    return df_preprocessed

def lower_tokenize_punct(lst:list[str]):
    stopwords_english = set(stopwords.words("english"))
    preprocessed_lst = []
    for sent in lst:
        lowercased_sent = sent.lower() # lowercase
        tokens_of_sent = word_tokenize(lowercased_sent) # tokenize
        clean_punctuation = [w for w in tokens_of_sent if w not in stopwords_english and w not in string.punctuation] # handle stopwords and punctuation

        preprocessed_lst.append(clean_punctuation)
    return preprocessed_lst
    
#lowercase, remove stopwords, tokenize --> Add new columns
def preprocess_dataF(df: pd.DataFrame) -> pd.DataFrame:
    df_out = df.copy(deep=True)
    premises = df["sentence1"].values
    hypotheses = df["sentence2"].values

    preprocessed_premises = lower_tokenize_punct(premises)
    preprocessed_hypotheses = lower_tokenize_punct(hypotheses)

    df_out["sentence1_preprocessed"] = preprocessed_premises
    df_out["sentence2_preprocessed"] = preprocessed_hypotheses

    return df_out

### Helpful Functions

In [50]:
def count_words_dict(df: pd.DataFrame) -> dict:
    out_count_dict = nltk.defaultdict(int)
    premises = df["sentence1_preprocessed"].values
    hypotheses = df["sentence2_preprocessed"].values

    merge = premises + hypotheses

    for sent in merge:
        for word in sent:
            out_count_dict[word] += 1
    return out_count_dict

def sort_dict(d: dict) -> dict:
    d_out = sorted(d.items(), key=lambda x: x[1] , reverse=True)
    return dict(d_out)

def remove_less_than_n_dict(d: dict, n:int) -> dict:
    new_dict = {k: v for k, v in d.items() if v >=n}
    return new_dict

In [21]:
len(snli_data_pd)

550152

In [22]:
subset_distinct = ["sentence1", "sentence2"]
snli_data_distinct = remove_duplicates(df=snli_data_pd, subset=subset_distinct)

In [23]:
len(snli_data_distinct)

549526

In [25]:
stopwords

TypeError: 'WordListCorpusReader' object is not iterable

In [26]:
a = []
b = [1, 2, 3]
c = [2, 3, 555]
a.append(b)
a.append(c)
a

[[1, 2, 3], [2, 3, 555]]

In [39]:
snli_data_pd_preprocessed = preprocess_dataF(snli_data_distinct)

In [41]:
snli_data_pd_preprocessed.columns

Index(['annotator_labels', 'captionID', 'gold_label', 'pairID', 'sentence1',
       'sentence1_binary_parse', 'sentence1_parse', 'sentence2',
       'sentence2_binary_parse', 'sentence2_parse', 'sentence1_preprocessed',
       'sentence2_preprocessed'],
      dtype='object')

In [44]:
snli_data_pd_preprocessed.head()

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse,sentence1_preprocessed,sentence2_preprocessed
0,[neutral],3416050480.jpg#4,neutral,3416050480.jpg#4r1n,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,A person is training his horse for a competition.,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,"[person, horse, jumps, broken, airplane]","[person, training, horse, competition]"
1,[contradiction],3416050480.jpg#4,contradiction,3416050480.jpg#4r1c,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is at a diner, ordering an omelette.",( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,"[person, horse, jumps, broken, airplane]","[person, diner, ordering, omelette]"
2,[entailment],3416050480.jpg#4,entailment,3416050480.jpg#4r1e,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is outdoors, on a horse.","( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...,"[person, horse, jumps, broken, airplane]","[person, outdoors, horse]"
3,[neutral],2267923837.jpg#2,neutral,2267923837.jpg#2r1n,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,They are smiling at their parents,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...,"[children, smiling, waving, camera]","[smiling, parents]"
4,[entailment],2267923837.jpg#2,entailment,2267923837.jpg#2r1e,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,There are children present,( There ( ( are children ) present ) ),(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...,"[children, smiling, waving, camera]","[children, present]"


In [43]:
snli_data_pd_preprocessed["sentence1_preprocessed"][0]

['person', 'horse', 'jumps', 'broken', 'airplane']

In [45]:
c_dict = count_words_dict(snli_data_pd_preprocessed)

In [46]:
c_dict

defaultdict(int,
            {'person': 35987,
             'horse': 7221,
             'jumps': 6628,
             'broken': 736,
             'airplane': 1219,
             'training': 734,
             'competition': 2594,
             'diner': 253,
             'ordering': 260,
             'omelette': 2,
             'outdoors': 9109,
             'children': 26545,
             'smiling': 8635,
             'waving': 1370,
             'camera': 11371,
             'parents': 1024,
             'present': 851,
             'kids': 10748,
             'frowning': 312,
             'boy': 58262,
             'jumping': 13405,
             'skateboard': 4720,
             'middle': 4580,
             'red': 43302,
             'bridge': 3216,
             'skates': 534,
             'sidewalk': 12819,
             'skateboarding': 1981,
             'trick': 3313,
             'wearing': 80985,
             'safety': 1904,
             'equipment': 2411,
             'older': 9949,


In [40]:
a = [[1, 2], [2, 5, 6]]
b = [[1, 4, 5, 6]]
a + b 

[[1, 2], [2, 5, 6], [1, 4, 5, 6]]

In [51]:
sorted_dict = sort_dict(c_dict)

In [52]:
sorted_dict_10 = remove_less_than_n_dict(d=sorted_dict, n=10)

In [55]:
sorted(sorted_dict_10.items(), key=lambda x: x[1])

[('reluctantly', 10),
 ('hitching', 10),
 ('laws', 10),
 ('jogged', 10),
 ('citrus', 10),
 ('threesome', 10),
 ('peole', 10),
 ('crowed', 10),
 ('summery', 10),
 ('jacked', 10),
 ('seventeen', 10),
 ('discovery', 10),
 ('fail', 10),
 ('sundae', 10),
 ('clasping', 10),
 ('loveseat', 10),
 ('rhino', 10),
 ('bishop', 10),
 ('flowerbed', 10),
 ('google', 10),
 ('proclaiming', 10),
 ('nodding', 10),
 ('chung', 10),
 ('wrecking', 10),
 ('lacing', 10),
 ('january', 10),
 ('boardroom', 10),
 ('drought', 10),
 ('pyramids', 10),
 ('discarded', 10),
 ('trellis', 10),
 ('etching', 10),
 ('carousal', 10),
 ('sturdy', 10),
 ('beared', 10),
 ('rc', 10),
 ('golfs', 10),
 ('horsing', 10),
 ('management', 10),
 ('strains', 10),
 ('hardcover', 10),
 ('majorette', 10),
 ('wearign', 10),
 ('four-way', 10),
 ('politicians', 10),
 ('manage', 10),
 ('conditioning', 10),
 ('intertube', 10),
 ('dunkin', 10),
 ('sheppard', 10),
 ('committing', 10),
 ('deadly', 10),
 ('em', 10),
 ('helment', 10),
 ('oustide', 10)